In [ ]:
import keras
import pickle
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
from sklearn import metrics
from scipy.stats import skew
from sklearn.base import clone
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.pipeline import Pipeline
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from sklearn.preprocessing import LabelEncoder
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_classification
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import make_classification
from tensorflow.keras.layers import Dense, Dropout
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report



MOUNTING CSV FILE

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


READING CSV FILE

In [ ]:

df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Assignment 3/CustomerChurn_dataset.csv', encoding='latin1')


EXPLORATORY DATA ANALYSIS

In [ ]:
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [ ]:
df.describe()

,SeniorCitizen,tenure,MonthlyCharges
count,7043.000000,7043.000000,7043.000000
mean,0.162147,32.371149,64.761692
std,0.368612,24.559481,30.090047
min,0.000000,0.000000,18.250000
25%,0.000000,9.000000,35.500000
50%,0.000000,29.000000,70.350000
75%,0.000000,55.000000,89.850000
max,1.000000,72.000000,118.750000


In [ ]:
df.shape

(7043, 21)

In [ ]:
df.columns

Index(['customerID', 'gender', 'SeniorCitizen', 'Partner', 'Dependents',
       'tenure', 'PhoneService', 'MultipleLines', 'InternetService',
       'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport',
       'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling',
       'PaymentMethod', 'MonthlyCharges', 'TotalCharges', 'Churn'],
      dtype='object')

DATA PREPROCESSING

In [ ]:
df.info()

In [ ]:
del df["customerID"]

In [ ]:
df['SeniorCitizen'] = df['SeniorCitizen'].astype(str)

In [ ]:
df['TotalCharges'] = df['TotalCharges'].replace(' ', np.NaN)
df['TotalCharges'] = df['TotalCharges'].astype(float)
df['TotalCharges'] = df['TotalCharges'].fillna(df['TotalCharges'].median())

In [ ]:
unique_values = {column: list(df[column].unique()) for column in df.select_dtypes(object).columns}
for key, value in unique_values.items():
    print(f'{key} : {value}')

In [ ]:
# Replace No internet sevice with No throughout
df['MultipleLines'] = df['MultipleLines'].replace('No phone service', 'No')
df[['OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
      'TechSupport', 'StreamingTV', 'StreamingMovies']] = df[['OnlineSecurity', 'OnlineBackup',
                                                                'DeviceProtection', 'TechSupport',
                                                                'StreamingTV', 'StreamingMovies']].replace('No internet service', 'No')

In [ ]:
#check for the missing values in the dataset
df.isnull().sum()

EDA

In [ ]:
df['Churn'].value_counts()

No     5174
Yes    1869
Name: Churn, dtype: int64

In [ ]:
sns.set_context('poster', font_scale= 0.6)
fig, ax  = plt.subplots(1, 3, figsize=(20, 6))

plt.suptitle('Histograms of all Numerical Features', fontsize = 20)

ax1 = sns.histplot(x = df['tenure'],color= 'teal', hue= df['Churn'], ax= ax[0], bins= 15, kde= True, palette= 'twilight')
ax1.set(xlabel= 'Tenure', ylabel= 'Frequency')

ax2 = sns.histplot(x = df['MonthlyCharges'], color= 'teal', hue= df['Churn'], ax= ax[1], bins= 15, kde= True, palette= 'winter')
ax2.set(xlabel= 'Monthly Charges', ylabel= 'Frequency')

ax3 = sns.histplot(x = df['TotalCharges'], color= 'teal', hue= df['Churn'], ax= ax[2], bins= 15, kde= True, palette= 'terrain')
ax3.set(xlabel= 'Total Charges', ylabel= 'Frequency')

plt.tight_layout()
plt.show()

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(15, 10))
plt.suptitle('Count plot for various categorical features', fontsize=30, color='teal')

sns.countplot(x='gender', data=df, hue='Churn', ax=ax[0, 0], palette='spring')
ax[0, 0].set_xlabel('Gender')

sns.countplot(x='SeniorCitizen', data=df, hue='Churn', ax=ax[0, 1], palette='winter')
ax[0, 1].set_xlabel('Senior Citizen')

sns.countplot(x='Partner', data=df, hue='Churn', ax=ax[1, 0], palette='summer')
ax[1, 0].set_xlabel('Partner')

sns.countplot(x='Dependents', data=df, hue='Churn', ax=ax[1, 1], palette='viridis')
ax[1, 1].set_xlabel('Dependents')

plt.tight_layout()
plt.show()


In [ ]:
# this creates countplot for me  to analyse the categorical variables
fig, ax = plt.subplots(3, 3, figsize=(20, 15))
plt.suptitle('Count plot for various categorical features', fontsize=30, color='teal')

sns.countplot(x='PhoneService', data=df, hue='Churn', ax=ax[0, 0], palette='spring')
ax[0, 0].set_xlabel('Phone Service')

sns.countplot(x='MultipleLines', data=df, hue='Churn', ax=ax[0, 1], palette='winter')
ax[0, 1].set_xlabel('Multiple Lines')

sns.countplot(x='InternetService', data=df, hue='Churn', ax=ax[0, 2], palette='summer')
ax[0, 2].set_xlabel('Internet Service')

sns.countplot(x='OnlineSecurity', data=df, hue='Churn', ax=ax[1, 0], palette='ocean')
ax[1, 0].set_xlabel('Online Security')

sns.countplot(x='OnlineBackup', data=df, hue='Churn', ax=ax[1, 1], palette='autumn')
ax[1, 1].set_xlabel('Online Backup')

sns.countplot(x='DeviceProtection', data=df, hue='Churn', ax=ax[1, 2], palette='YlGnBu')
ax[1, 2].set_xlabel('Device Protection')

sns.countplot(x='TechSupport', data=df, hue='Churn', ax=ax[2, 0], palette='copper')
ax[2, 0].set_xlabel('Tech Support')

sns.countplot(x='StreamingTV', data=df, hue='Churn', ax=ax[2, 1], palette='plasma')
ax[2, 1].set_xlabel('Streaming TV')

sns.countplot(x='StreamingMovies', data=df, hue='Churn', ax=ax[2, 2], palette='bone')
ax[2, 2].set_xlabel('Streaming Movies')

plt.tight_layout()
plt.show()

In [ ]:
correlation = df.corr().round(2)
plt.figure(figsize = (14,7))
sns.heatmap(correlation, annot = True, cmap = 'YlOrBr')

ENCODING

In [ ]:

binary_columns = ['gender', 'Partner', 'Dependents', 'PhoneService', 'MultipleLines',
                  'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport',
                  'StreamingTV', 'StreamingMovies', 'PaperlessBilling']

le = LabelEncoder()
for column in binary_columns:
    df[column] = le.fit_transform(df[column])

In [ ]:
dummies_payment = pd.get_dummies(df['PaymentMethod'], drop_first= True)
data = pd.concat([df, dummies_payment], axis=1)
data.drop('PaymentMethod',axis =1 , inplace= True)

In [ ]:
internet_ordering = ['No', 'DSL', 'Fiber optic']
contract_ordering = ['Month-to-month', 'One year', 'Two year']

data['InternetService'] = data['InternetService'].apply(lambda x: internet_ordering.index(x))
data['Contract'] = data['Contract'].apply(lambda x: contract_ordering.index(x))

In [ ]:
skew_df = pd.DataFrame(data.select_dtypes(np.number).columns, columns=['Feature'])
skew_df['Skew'] = skew_df['Feature'].apply(lambda feature: skew(data[feature]))
skew_df['Absolute Skew'] = skew_df['Skew'].apply(abs)
skew_df

SCALING AND SPLITTING

In [ ]:
#Scaling
X = data.drop('Churn', axis= 1)
y = data['Churn']

sc = StandardScaler()
X = sc.fit_transform(X)

#Spilting
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.2, stratify= y, random_state= 42)
print(y_train.value_counts())
print(y_test.value_counts(

))

In [ ]:
X_train.shape

In [ ]:
# Select relevant features
selected_features = ['Contract', 'OnlineSecurity', 'OnlineBackup', 'TechSupport', 'PaymentMethod', 'tenure', 'MonthlyCharges', 'TotalCharges', 'gender', 'SeniorCitizen', 'Partner', 'Dependents', 'Churn']
df = df[selected_features]

# Convert categorical columns to numerical using one-hot encoding
categorical_columns = ['Contract', 'OnlineSecurity', 'OnlineBackup', 'TechSupport', 'PaymentMethod', 'gender', 'Partner', 'Dependents']
df = pd.get_dummies(df, columns=categorical_columns, drop_first=True)

# Convert 'TotalCharges' to numeric and handle missing values
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
df = df.dropna()


MODEL TRAINING WITH FUNCTIONAL API

In [ ]:
 # Assuming 'Churn' column contains 'Yes' and 'No'
df['Churn'] = df['Churn'].map({'Yes': 1, 'No': 0})


label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_test = label_encoder.transform(y_test)

input_shape = X_train.shape[1]

# Build the model using the Functional API
input_layer = Input(shape=(input_shape,))
hidden_layer1 = Dense(64, activation='relu')(input_layer)
dropout_layer1 = Dropout(0.5)(hidden_layer1)
hidden_layer2 = Dense(32, activation='relu')(dropout_layer1)
dropout_layer2 = Dropout(0.5)(hidden_layer2)
output_layer = Dense(1, activation='sigmoid')(dropout_layer2)

model = Model(inputs=input_layer, outputs=output_layer)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=64, validation_data=(X_test, y_test))

# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_accuracy}")

HYPERTUNING WITH GRIDSEARCH

In [ ]:

#  hyperparameters for grid search
learning_rates = [0.001, 0.01, 0.1]
dropouts = [0.5]
epochs_list = [50, 100]  # Include epochs to search
hidden_layer_sizes = [32, 64, 128]  # Include hidden layer sizes to search

best_accuracy = 0
best_auc = 0
best_params = {}

for lr in learning_rates:
    for dropout_rate in dropouts:
        for epochs in epochs_list:
            for hidden_size in hidden_layer_sizes:
                # Build the model
                input_layer = Input(shape=(input_shape,))
                hidden_layer1 = Dense(hidden_size, activation='relu')(input_layer)
                dropout_layer1 = Dropout(dropout_rate)(hidden_layer1)
                hidden_layer2 = Dense(hidden_size // 2, activation='relu')(dropout_layer1)
                dropout_layer2 = Dropout(dropout_rate)(hidden_layer2)
                output_layer = Dense(1, activation='sigmoid')(dropout_layer2)

                model = Model(inputs=input_layer, outputs=output_layer)

                # Compile the model
                opt = Adam(learning_rate=lr)
                model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

                # Train the model
                history = model.fit(X_train, y_train, epochs=epochs, batch_size=64, validation_data=(X_test, y_test), verbose=0)

                # Evaluate the model on the test set
                y_pred_prob = model.predict(X_test)
                y_pred = (y_pred_prob > 0.5).astype(int)
                accuracy = accuracy_score(y_test, y_pred)
                auc = roc_auc_score(y_test, y_pred_prob)

                # Keep track of the best accuracy, AUC, and hyperparameters
                if accuracy > best_accuracy:
                    best_accuracy = accuracy
                    best_auc = auc
                    best_params = {'learning_rate': lr, 'dropout_rate': dropout_rate, 'epochs': epochs, 'hidden_size': hidden_size}


45/45 [==============================] - 0s 2ms/step


ACCURACY AND AUC SCORE

In [ ]:

# Output the best parameters, best accuracy, and best AUC found
print("Best accuracy:", best_accuracy)
print("Best AUC:", best_auc)
print("Best hyperparameters:", best_params)


Best accuracy: 0.7963094393186657
Best AUC: 0.8380144152522668
Best hyperparameters: {'learning_rate': 0.001, 'dropout_rate': 0.5, 'epochs': 50, 'hidden_size': 128}


SCALER MODEL

In [ ]:
# Select numerical features
numerical_features = ['tenure', 'MonthlyCharges', 'TotalCharges']
# Create scaler
scaler = StandardScaler()
# Fit and transform features
df[numerical_features] = scaler.fit_transform(df[numerical_features])

# Save scaler
with open('scaler.pkl', 'wb') as scaler_file:
    pickle.dump(scaler, scaler_file)


LABEL ENCODER MODEL

In [ ]:
# Assuming 'df' is your dataset
label_encoder = LabelEncoder()

# Fit and transform the 'Churn' column
df['Churn'] = label_encoder.fit_transform(df['Churn'])

# Save the label encoder
with open('label_encoder.pkl', 'wb') as label_encoder_file:
    pickle.dump(label_encoder, label_encoder_file)


SAVE MODEL

In [ ]:
# Save the best model using
model.save('best_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
